In [6]:
import tkinter as tk
from tkinter import filedialog, messagebox, Toplevel
from PIL import Image, ImageTk
import cv2
import numpy as np
import tensorflow as tf
from mtcnn import MTCNN
import threading
import time

# Classifier class to handle model loading and predictions
class Classifier:
    def __init__(self, model_path):
        self.model = tf.saved_model.load(model_path)
        self.predict_fn = self.create_predict_function()

    def create_predict_function(self):
        @tf.function
        def predict_function(features):
            return self.model.signatures['serving_default'](input_1=features)
        return predict_function

    def predict(self, features):
        return self.predict_fn(features)

class AgeClassifier:
    def __init__(self, model_path):
        self.model = tf.saved_model.load(model_path)
        self.predict_fn = self.model.signatures['serving_default']

    def predict(self, features):
        output = self.predict_fn(keras_tensor_13=features)
        predicted_age = output['output_0']
        print(predicted_age)
        return predicted_age

# Function to detect vehicles
def detect_vehicle(image, classifier):
    img_resized = cv2.resize(image, (416, 416))
    img_array = np.expand_dims(img_resized, axis=0).astype(np.float32) / 255.0
    predictions = classifier.predict(img_array)
    vehicle_labels = {0: 'Ambulance', 1: 'Bus', 2: 'Car', 3: 'Motorcycle', 4: 'Truck'}
    class_index = np.argmax(predictions)
    return vehicle_labels[class_index] == 'Car'

# Function to detect faces using MTCNN
def detect_faces(image):
    detector = MTCNN()
    faces = detector.detect_faces(image)
    bounding_boxes = [face['box'] for face in faces]
    return bounding_boxes, len(faces)

# Function to predict sleeping
def predict_sleeping(face, classifier):
    face_resized = cv2.resize(face, (224, 224))
    face_array = np.expand_dims(face_resized, axis=0).astype(np.float32) / 255.0
    predictions = classifier.predict(face_array)
    sleeping_labels = {0: 'Drowsy', 1: 'Awake'}
    class_index = np.argmax(predictions)
    return sleeping_labels[class_index] == 'Drowsy'

# Function to predict age
def predict_age(face, classifier):
    face_resized = cv2.resize(face, (128, 128))
    face_array = np.expand_dims(face_resized, axis=0).astype(np.float32) / 255.0
    face_array = face_array[:, :, :, 0:1]  # Extract the first channel and keep the shape (1, 128, 128, 1)
    predicted_age = classifier.predict(face_array)[0][0]
    return predicted_age

# Function to process image or video
def process_media(input_path, vehicle_classifier, sleeping_classifier, age_classifier, video_label=None, is_video=False):
    if is_video:
        cap = cv2.VideoCapture(input_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter('processed_video.mp4', fourcc, 30, (640, 480))
        
        num_sleeping = 0
        
        def process_frames():
            nonlocal num_sleeping
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                frame = cv2.resize(frame, (640, 480))
                
                # Detect vehicle
                if not detect_vehicle(frame, vehicle_classifier):
                    continue
                
                # Detect faces using MTCNN
                faces, num_faces = detect_faces(frame)
                print(f"Number of faces detected: {num_faces}")
                
                # Count the number of sleeping persons
                for (x, y, w, h) in faces:
                    face = frame[y:y+h, x:x+w]
                    # Predict sleeping
                    if predict_sleeping(face, sleeping_classifier):
                        num_sleeping += 1
                        # Predict age
                        age = int(predict_age(face, age_classifier))
                        label = f"Age:{age}"
                        # Draw rectangle around the face
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                        # Display label
                        cv2.putText(frame, label, (x, y - 15), cv2.FONT_HERSHEY_TRIPLEX, 0.7, (0, 0, 255), 1)

                # Convert frame to PhotoImage
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_pil = Image.fromarray(frame_rgb)
                frame_tk = ImageTk.PhotoImage(image=frame_pil)
                video_label.configure(image=frame_tk)
                video_label.image = frame_tk
                
                # Write the processed frame to the video file
                out.write(frame)
                
                time.sleep(0.033) 
        
        video_thread = threading.Thread(target=process_frames)
        video_thread.start()

        while video_thread.is_alive():
            video_thread.join(0.1)
        
        cap.release()
        out.release()
        
        return 'processed_video.mp4', num_sleeping
    else:
        frame = cv2.imread(input_path)
        
        # Detect vehicle
        if not detect_vehicle(frame, vehicle_classifier):
            messagebox.showinfo("Info", "No car detected in the image. Processing stopped.")
            return None, 0
        
        faces, num_faces = detect_faces(frame)
        
        num_sleeping = 0
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            # Predict sleeping
            if predict_sleeping(face, sleeping_classifier):
                num_sleeping += 1
                # Predict age
                age = int(predict_age(face, age_classifier))
                label = f"Age:{age}"
                # Draw rectangle around the face
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                # Display label
                cv2.putText(frame, label, (x, y - 15), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 0, 255), 1)
        
        # Save the processed image
        cv2.imwrite('processed_image.jpg', frame)
        
        return 'processed_image.jpg', num_sleeping

# Tkinter GUI
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Sleep Detection")
        
        self.label = tk.Label(root, text="Upload an image or video:")
        self.label.pack(pady=20)
        
        self.image_button = tk.Button(root, text="Upload Image", command=self.upload_image)
        self.image_button.pack(side=tk.LEFT, padx=10)
        
        self.video_button = tk.Button(root, text="Upload Video", command=self.upload_video)
        self.video_button.pack(side=tk.RIGHT, padx=10)
        
        self.image_label = tk.Label(root)
        self.image_label.pack(pady=20)
        
        self.sleeping_persons_label = tk.Label(root, font=("Arial", 12))
        self.sleeping_persons_label.pack(pady=10)

        # Load models using Classifier class
        self.vehicle_classifier = Classifier('Vehicle Classification Model')
        self.sleeping_classifier = Classifier('Sleeping_Detection_Model')
        self.age_classifier = AgeClassifier('Age Prediction Model')

    def upload_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            self.process_image(file_path)

    def upload_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
        if file_path:
            self.process_video(file_path)

    def process_image(self, file_path):
        messagebox.showinfo("Info", f"Processing image: {file_path}. Please wait.")
        processed_frame, num_sleeping = process_media(file_path, self.vehicle_classifier, self.sleeping_classifier, self.age_classifier, is_video=False)
        if processed_frame:
            self.display_image(processed_frame, num_sleeping)
        else:
            self.image_label.configure(image='')
            self.image_label.image = None
            self.sleeping_persons_label.configure(text="No car detected.")

    def process_video(self, file_path):
        messagebox.showinfo("Info", f"Processing video: {file_path}. Please wait.")
        video_window = Toplevel(self.root)
        video_window.title("Video Output")
        video_label = tk.Label(video_window)
        video_label.pack()

        def update_frame():
            cap = cv2.VideoCapture(file_path)
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Detect vehicle
                if detect_vehicle(frame, self.vehicle_classifier):
                    # Detect faces using MTCNN
                    faces, num_faces = detect_faces(frame)
                    print(f"Number of faces detected: {num_faces}")
                    
                    # Count the number of sleeping persons
                    for (x, y, w, h) in faces:
                        face = frame[y:y+h, x:x+w]
                        # Predict sleeping
                        if predict_sleeping(face, self.sleeping_classifier):
                            # Predict age
                            age = int(predict_age(face, self.age_classifier))
                            label = f"Age:{age}"
                            # Draw rectangle around the face
                            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                            # Display label
                            cv2.putText(frame, label, (x, y - 15), cv2.FONT_HERSHEY_TRIPLEX, 0.7, (0, 0, 255), 1)
                
                # Convert frame to PhotoImage
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_pil = Image.fromarray(frame_rgb)
                frame_tk = ImageTk.PhotoImage(image=frame_pil)
                video_label.configure(image=frame_tk)
                video_label.image = frame_tk

        threading.Thread(target=update_frame).start()

    def display_image(self, file_path, num_sleeping):
        image_pil = Image.open(file_path)
        image_tk = ImageTk.PhotoImage(image=image_pil)
        
        self.image_label.configure(image=image_tk)
        self.image_label.image = image_tk
        
        self.sleeping_persons_label.configure(text=f"Number of sleeping persons: {num_sleeping}")

# Create the main Tkinter window
root = tk.Tk()
app = App(root)
root.mainloop()

1/1 [==============================] - 0s 141ms/step
tf.Tensor([[31.075558]], shape=(1, 1), dtype=float32)
1/1 [==============================] - 0s 156ms/step
tf.Tensor([[30.652832]], shape=(1, 1), dtype=float32)
tf.Tensor([[29.534634]], shape=(1, 1), dtype=float32)
tf.Tensor([[29.495617]], shape=(1, 1), dtype=float32)


Exception in thread Thread-9 (update_frame):
Traceback (most recent call last):
  File "C:\Users\Sidhi\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\Sidhi\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Sidhi\AppData\Local\Temp\ipykernel_7284\4200493759.py", line 246, in update_frame
  File "C:\Users\Sidhi\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1675, in configure
    return self._configure('configure', cnf, kw)
  File "C:\Users\Sidhi\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1665, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))
_tkinter.TclError: invalid command name ".!toplevel.!label"
